## 29-31 July 2019 Melt Event at Summit Station, Greenland

In [4]:
def readNOAAsfcmet(year,month):
    from   datetime import datetime
    import pandas   as     pd
    
    # Read a surface meteorology file from NOAA's site at Summit Station, Greenland.
    df   = pd.read_csv('/Users/vonw/data/summit/sfcmet/noaamet/met_sum_insitu_1_obop_minute_' + str(year) + '_' + str(month).zfill(2) + '.txt', delim_whitespace=True, parse_dates=True, names=['site','year','month','day','hour','minute','wind direction','wind speed','wind steadiness factor','barometric pressure','temperature at 2 meters','temperature at 10 meters','temperature at tower top','relative humidity','precipitation intensity'])
    
    #  Convert string data to numeric values
    df['year'].apply(int);
    df['month'].apply(int);
    df['day'].apply(int);
    df['hour'].apply(int);
    df['minute'].apply(int);
    df['barometric pressure'].apply(float);
    df['wind direction'].apply(float);
    df['wind speed'].apply(float);
    df['wind steadiness factor'].apply(float);
    df['temperature at 2 meters'].apply(float);
    df['temperature at 10 meters'].apply(float);
    df['temperature at tower top'].apply(float);
    df['relative humidity'].apply(float);
    df['precipitation intensity'].apply(float);
    
    # Create a time index.
    time = [datetime(t[0],t[1],t[2],t[3],t[4],0) for t in zip(df.year,df.month,df.day,df.hour,df.minute)] 
    df.index = time
    
    return df

In [5]:
import pandas as pd
from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.palettes import Viridis3
from bokeh.plotting import figure
from rad2bt import rad2bt

In [6]:
output_notebook()

Loading BokehJS ...

In [8]:
from readDMV import readDMV
d = '/Users/vonw/data/paeri/raw/AE'
c1_30 = readDMV(d + '190730' + '/' + '190730' + 'C1.RNC')
c1_31 = readDMV(d + '190731' + '/' + '190731' + 'C1.RNC')

In [10]:
noaa = readNOAAsfcmet(2019,7)

In [11]:
p1=figure(plot_width=900,
       plot_height=200,
       x_axis_type='datetime',
       x_axis_label='Time (UTC)',
       x_range=[pd.datetime(2019,7,30,0,0,0), pd.datetime(2019,8,1,0,0,0)],
       y_axis_label='Radiance [mW m-2 sr-1 (cm-1)-1]',
       title='962-cm-1 microwindow radiance, 30-31 July 2019')
p1.line(c1_30.time.values, c1_30.mean_rad[:,957].values, color=Viridis3[0])
p1.line(c1_31.time.values, c1_31.mean_rad[:,957].values, color=Viridis3[1])

p2=figure(plot_width=900,
       plot_height=200,
       x_range=p1.x_range,
       x_axis_type='datetime',
       x_axis_label='Time (UTC)',
       y_axis_label='Radiance [mW m-2 sr-1 (cm-1)-1]',
       title='Outside Temperatures, 30-31 July 2019')
p2.line(c1_30.time.values,c1_30.airNearBBsTemp.values-273.15, color='tomato', legend='airNearBBsTemp')
p2.line(c1_31.time.values,c1_31.airNearBBsTemp.values-273.15, color='tomato')
p2.line(c1_30.time.values,c1_30.outsideAirTemp.values-273.15, color=Viridis3[2], legend='outsideAirTemp')
p2.line(c1_31.time.values,c1_31.outsideAirTemp.values-273.15, color=Viridis3[2])
p2.line(noaa['2019-07-30'].index, noaa['2019-07-30']['temperature at 2 meters'].values, color=Viridis3[0], legend='NOAA 2-m Temp')
p2.line(noaa['2019-07-31'].index, noaa['2019-07-31']['temperature at 2 meters'].values, color=Viridis3[1])
p2.line([c1_30.time[0].values, c1_31.time[-1].values], [0., 0.], color='blue', line_dash='dashed')

p3=figure(plot_width=900,
       plot_height=200,
       x_range=p1.x_range,
       x_axis_type='datetime',
       x_axis_label='Time (UTC)',
       y_axis_label='Radiance [mW m-2 sr-1 (cm-1)-1]',
       title='Brightness Temperatures, 30-31 July 2019')
p3.line(c1_30.time.values, rad2bt(c1_30.wnum1[374:395].mean(), c1_30.mean_rad[:,374:395].mean(axis=1)).values-273.15, color=Viridis3[0], legend='670-680 cm-1')
p3.line(c1_31.time.values, rad2bt(c1_31.wnum1[374:395].mean(), c1_31.mean_rad[:,374:395].mean(axis=1)).values-273.15, color=Viridis3[1])
p3.line(c1_30.time.values, rad2bt(c1_30.wnum1[957], c1_30.mean_rad[:,957]).values-273.15, color=Viridis3[0], legend='962 cm-1')
p3.line(c1_31.time.values, rad2bt(c1_31.wnum1[957], c1_31.mean_rad[:,957]).values-273.15, color=Viridis3[1])
p3.line([c1_30.time[0].values, c1_31.time[-1].values], [0., 0.], color='blue', line_dash='dashed')

layout = gridplot([[p1], [p2], [p3]])
show(layout)

/Users/vonw/anaconda3/envs/work/lib/python3.7/site-packages/xarray/core/computation.py:564: RuntimeWarning: invalid value encountered in log
  result_data = func(*input_data)


In [12]:
c1_30.wnum1[957]

<xarray.DataArray 'wnum1' ()>
array(961.40155)
Coordinates:
    wnum1    float64 961.4

In [13]:
p4=figure(plot_width=900,
       plot_height=600,
       x_axis_label='Window Radiance [mW m-2 sr-1 (cm-1)-1]',
       y_axis_label='CO2 Brightness Temperature (K)',
       title='Window radiance vs. Outside Air Temperature, 30-31 July 2019')
p4.scatter(c1_30.mean_rad[:,957].values, c1_30.outsideAirTemp.values-273.15, color=Viridis3[0])
p4.scatter(c1_31.mean_rad[:,957].values, c1_31.outsideAirTemp.values-273.15, color=Viridis3[1])

show(p4)

In [14]:
p4=figure(plot_width=900,
       plot_height=600,
       x_axis_label='Window Radiance [mW m-2 sr-1 (cm-1)-1]',
       y_axis_label='CO2 Brightness Temperature (K)',
       title='Window radiance vs. CO2 Brightness Temperature, 30-31 July 2019')
p4.scatter(c1_30.mean_rad[:,957].values, rad2bt(c1_30.wnum1[374:395].mean(), c1_30.mean_rad[:,374:395].mean(axis=1)).values-273.15, color=Viridis3[0])
p4.scatter(c1_31.mean_rad[:,957].values, rad2bt(c1_31.wnum1[374:395].mean(), c1_31.mean_rad[:,374:395].mean(axis=1)).values-273.15, color=Viridis3[1])

show(p4)